In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import math
import torch
import gpytorch
import pandas as pd
import random
import scipy.stats
import numpy as np
import tqdm.notebook
import time
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from matplotlib import pyplot as plt
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from torch.utils.data import TensorDataset, DataLoader

from gpytorch import Module

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Extracting datasets
datasets = ['boston', 'concrete', 'yacht', 'wine']

# data(d), original features(x_), original target(y_), scale of test split(s), low of test split(l), normalized features(x), normalized target(y)
d, x_, y_, s, l, x, y = {}, {}, {}, {}, {}, {}, {}

#min_max_scaler = preprocessing.MinMaxScaler()

for i in range(len(datasets)):
    d[i] = pd.read_csv('data_vrbound/{}/data.txt'.format(datasets[i]), 
                       delim_whitespace=True, header=None).dropna().astype(float)
    x_[i] = d[i].iloc[:,0:-1]
    y_[i] = d[i].iloc[:,-1]

In [3]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [5]:
def split_data(x, y, test_split, seed, norm=True):
    
    x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=test_split, random_state=seed)
    
    mean_y_train = y_train.mean()
    std_y_train = y_train.std()
    
    if norm:
        x_train = pd.DataFrame(x_train.values)
        x_test = pd.DataFrame(x_test.values)
        y_train = pd.DataFrame(y_train.values.reshape(-1,1))
        y_test = pd.DataFrame(y_test.values.reshape(-1,1))

        mean_x_train = x_train.mean()
        std_x_train = x_train.std()
        std_x_train[std_x_train==0] = 1
        x_train = (x_train - mean_x_train) / std_x_train
        x_test = (x_test - mean_x_train) / std_x_train
        y_train = (y_train - mean_y_train) / std_y_train

    else:
        x_train = pd.DataFrame(x_train.values)
        x_test = pd.DataFrame(x_test.values)
        y_train = pd.DataFrame(y_train.values.reshape(-1,1))
        y_test = pd.DataFrame(y_test.values.reshape(-1,1))
        scale = 1
        low = 0

    list_of_tensors = [torch.tensor(x_train[i], dtype=torch.float32) for i in x_train]
    train_x = torch.stack(list_of_tensors)
    train_x = torch.transpose(train_x, 0, 1)

    list_of_tensors = [torch.tensor(x_test[i], dtype=torch.float32) for i in x_test]
    test_x = torch.stack(list_of_tensors)
    test_x = torch.transpose(test_x, 0, 1)

    train_y = torch.tensor(y_train[0], dtype=torch.float32)
    test_y = torch.tensor(y_test[0], dtype=torch.float32)
    
#     return train_x, test_x, train_y, test_y, scale, low 

    return train_x, test_x, train_y, test_y, mean_y_train, std_y_train

split_data(x_[2], y_[2], test_split=0.1, seed=1, norm=True)

(tensor([[ 0.0782,  1.5890,  1.2293,  0.4232,  1.2083,  1.3469],
         [ 0.0782,  0.2379, -0.0514, -1.6154,  1.7331,  1.5939],
         [ 0.0120,  0.4558, -1.7324,  0.0444, -1.8193, -0.8754],
         ...,
         [ 0.0120,  0.1943, -1.8124, -1.7237, -0.2450, -1.1223],
         [ 1.6009,  1.5890, -0.0514,  0.5495, -0.2450,  1.1000],
         [ 0.0782,  0.0636, -0.0514,  2.5521, -1.8193,  0.6062]]),
 tensor([[ 0.0120,  0.1943, -1.8124, -1.7237, -0.2450, -0.8754],
         [ 0.0782,  1.5890, -1.8124,  0.5315, -1.9404,  0.8531],
         [ 1.6009,  1.5890, -0.0514,  0.5495, -0.2450, -0.6285],
         [-1.7094,  1.5890, -0.0514,  0.5495, -0.2450, -1.6162],
         [ 1.6009, -1.4619, -0.0514, -0.3345, -0.2450,  0.1123],
         [ 1.6009,  0.0636,  1.2293,  0.0083,  1.2083,  0.8531],
         [ 0.0782, -1.4619, -0.1315, -0.4608, -0.2046, -1.3692],
         [ 1.6009,  1.5890, -0.0514,  0.5495, -0.2450, -0.8754],
         [-1.7094, -1.4619, -0.0514, -0.3345, -0.2450,  0.3592],
         

In [6]:
class GP(Module):
    def __init__(self, likelihood, model, num_data):
        super().__init__()
        self.model = model
        self.likelihood = likelihood
        self.num_data = num_data
        self.beta = beta

    def _log_likelihood_term(self, variational_dist_f, target, **kwargs):
        return self.likelihood.expected_log_prob(target, variational_dist_f, **kwargs).sum(-1)
    
    def forward(self, approximate_dist_f, target, **kwargs):
        num_batch = approximate_dist_f.event_shape[0]
        log_likelihood = self._log_likelihood_term(approximate_dist_f, target, **kwargs).div(num_batch)
        kl_divergence = self.model.variational_strategy.kl_divergence().div(self.num_data / self.beta)
        
        # Add any additional registered loss terms
        added_loss = torch.zeros_like(log_likelihood)
        had_added_losses = False
        for added_loss_term in self.model.added_loss_terms():
            added_loss.add_(added_loss_term.loss())
            had_added_losses = True
        
        log_prior = torch.zeros_like(log_likelihood)
        for name, module, prior, closure, _ in self.named_priors():
            log_prior.add_(prior.log_prob(closure(module)).sum().div(self.num_data))

        return log_likelihood - kl_divergence + log_prior - added_loss

In [13]:
def svgp(x, y, training_iter, test_split, seed, printout):
    
    torch.manual_seed(seed)
    
    start = time.process_time()
    train_x, test_x, train_y, test_y, loc, scale = split_data(x, y, test_split, seed, norm=True)
    end = time.process_time()
    
    num_inducing = 200
    class GPModel(gpytorch.models.ApproximateGP):
        def __init__(self, inducing_points):
            variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
            variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
            super(GPModel, self).__init__(variational_strategy)
            self.mean_module = gpytorch.means.ZeroMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    inducing_points = train_x[:num_inducing]
#         inducing_points = np.random.choice(train_x, size=num_inducing, replace=False)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(inducing_points=inducing_points)
    
    model.train()
    likelihood.train()
    
    params = model.parameters()
        
    optimizer = torch.optim.Adam(params, lr=0.1)
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

    for i in range(training_iter):
        optimizer.zero_grad()
        train_pred = model(train_x)
        train_res_pred = gpytorch.distributions.MultivariateNormal(train_pred.mean, train_pred._covar)
        
        lld = -mll(train_pred, train_y)
        lld.backward()
        optimizer.step()
            

    model.eval()
    likelihood.eval()

    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        preds1 = model(test_x)
        observed_pred = preds1

        scaled_pred = gpytorch.distributions.MultivariateNormal(observed_pred.mean*scale + loc, observed_pred._covar*scale**2)
        
        lld = likelihood.log_marginal(test_y, scaled_pred).mean()
    
    
    raw_rmse = torch.sqrt(torch.mean(torch.pow((scaled_pred.mean - test_y) , 2)))
    
    if printout:
        print(observed_pred)
        print(test_x.size())
        print(test_y.size())
        print('pred_double: {}'.format(observed_pred.mean[0]))
        print('scaled_pred: {}'.format(scaled_pred.mean[0]))
        print('y_test: {}'.format(test_y[0]))
        print('time: {}'.format(end-start))

    
    return lld, raw_rmse

training_iter = 100
test_split = 0.1
seed = 5
printout=True

svgp(x_[0], y_[0], training_iter, test_split, seed, printout)

MultivariateNormal(loc: torch.Size([51]))
torch.Size([51, 13])
torch.Size([51])
pred_double: 2.1274304389953613
scaled_pred: 42.18560028076172
y_test: 37.599998474121094
time: 0.007792999999999495


(tensor(-2.5145), tensor(2.7270))

In [18]:
res = {}
# n = len(d)
n_dataset = [0,1,2,3] # max 4
itr = 30

for j in n_dataset:
    res[j] = {}
    a_1, a_2, a_3, a_4, t, a_5 = [], [], [], [], [], []
    
    start = time.process_time()
    for i in range(itr):
        start = time.process_time()
        loss, raw_rmse = svgp(
            x_[j], y_[j], training_iter=1, 
            test_split=0.1, 
            seed=i, printout=False)
        end = time.process_time()
        
        t.append(end-start)
        a_1.append(loss)
        a_2.append(raw_rmse)
        
    res[j]['a_1'] = a_1
    res[j]['a_2'] = a_2
    
    
    print('Dataset :{}'.format(datasets[j]))
    print('loss: {}, std:{}'.format(mean_confidence_interval(a_1), np.std(a_1)))
    print('raw_rmse: {}, std:{}'.format(mean_confidence_interval(a_2), np.std(a_2)))
    print('Time: {} seconds, std:{}'.format(mean_confidence_interval(t), np.std(t)))
    print('------------------------------------')



Dataset :boston
loss: (-3.682078, -3.7304123408918355, -3.633743428456309), std:0.12726639211177826
raw_rmse: (9.002504, 8.714967297232654, 9.290041400277111), std:0.7570955753326416
Time: (0.1730533666666659, 0.15893877244856397, 0.18716796088476784) seconds, std:0.03716424526834373
------------------------------------
Dataset :concrete
loss: (-4.2178626, -4.254858240808667, -4.180866971288501), std:0.09741086512804031
raw_rmse: (15.804794, 15.39264671296252, 16.216941910084355), std:1.0851998329162598
Time: (0.23534340000000023, 0.22723755977230176, 0.2434492402276987) seconds, std:0.021342975198099894
------------------------------------
Dataset :yacht
loss: (-4.0810246, -4.1531010019995405, -4.008948291518526), std:0.18977968394756317
raw_rmse: (13.791112, 12.860289758794416, 14.721934133417498), std:2.4508891105651855
Time: (0.12385276666666697, 0.12040934334730508, 0.12729618998602887) seconds, std:0.00906666014098911
------------------------------------
Dataset :wine
loss: (-1.2